# Húsnæðislánareiknivél

Þetta er lauflétt húsnæðislánareiknivél sem ég bjó til til þess að skilja betur hvernig húsnæðislán virka.

Áður en þú byrjar að nota reiknivélina þarftu að setja upp þau forritasöfn sem eru nauðsynleg. Það er gert með því að keyra `python -m pip install -r requirements.txt`. Þessi skrifta hefur verið prófuð á Python útgáfu 3.11.2.

Til að nota reiknivélina er nóg breyta breytunum í næsta hólfi (Stillingar) og keyra reiknivélina (Run all).

## Verkefnalisti:

- [ ] Útfæra verðtryggð lán
- [ ] Yfirlit yfir tölfræði lána (t.d. heildarkostnað, lægsta, hæsta og miðgildi afborgunar)
- [ ] Gröf og yfirlit yfir eignamyndun

In [ ]:
# Stillingar - Hægt að breyta þessum gildum

# Sameiginlegar stillingar fyrir öll lán
lansupphaed: int = 20000000

# Óverðtryggð lán
lanstimi_overdtryggt: int = 40
vextir_overdtryggt: float = 0.0925  # Ársvextir

# Verðtryggð lán
verdbolga: float = 0.0981  # Áætluð árleg
lanstimi_verdtryggt: int = 15
vextir_verdtryggt: float = 0.0265  # Ársvextir

# Fasteign
verd: int = 60000000
fasteignamat: int = 50000000

In [ ]:
# Afleiddar breytur - Ekki breyta

fjoldi_afborgana_overdtryggt = 12 * lanstimi_overdtryggt
fjoldi_afborgana_verdtryggt = 12 * lanstimi_verdtryggt

In [ ]:
# Innfluttningur á kóðasöfnum

from typing import Generator

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reikniföll

def jafnar_afborganir(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
) -> Generator[tuple[int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls

    To-do: Útfæra verðtryggingu.
    '''
    afborgun_af_hofudstol = lansupphaed / fjoldi_afborgana
    hofudstoll = float(lansupphaed)
    for i in range(1, fjoldi_afborgana + 1):
        vaxtagreidsla = hofudstoll * (vextir) / 12
        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla
        
        yield (
            i,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            int(round(heildargreidsla, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_afborganir_vertryggt(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    verdbolga: float,
) -> Generator[tuple[int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls

    To-do: Útfæra verðtryggingu.
    '''
    afborgun_af_hofudstol = lansupphaed / fjoldi_afborgana

    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum því að umreikna
    # hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu
    # 1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    # m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    hofudstoll = float(lansupphaed)
    for i in range(1, fjoldi_afborgana + 1):
        verdbaetur =  hofudstoll * manadarleg_verdbolga
        hofudstoll += verdbaetur

        vaxtagreidsla = hofudstoll * (vextir / 12)
        afborgun_af_hofudstol = hofudstoll / (fjoldi_afborgana - i + 1) # afborgun_af_hofudstol * (1 + ((verdbolga / 100) / 12))
        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
        i,
        int(round(afborgun_af_hofudstol, 0)),
        int(round(vaxtagreidsla, 0)),
        int(round(heildargreidsla, 0)),
        int(round(verdbaetur, 0)),
        int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol

def jafnar_greidslur(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
) -> Generator[tuple[int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur láns með jöfnum greidslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - heildar greiðsla
        - stöðu höfuðstóls

    To-do: Útfæra verðtryggingu.
    '''
    hofudstoll = float(lansupphaed)

    r = vextir / 12
    n = fjoldi_afborgana
    heildargreidsla = lansupphaed * (r * (1 + r)**n) / ((1 + r)**n - 1)

    for i in range(1, fjoldi_afborgana + 1):
        vaxtagreidsla = hofudstoll * r
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        yield (
            i,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            int(round(heildargreidsla, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol



In [ ]:
# Útreikningur

jafnar_afborganir_overdtryggt = pd.DataFrame(
    jafnar_afborganir(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
        vextir=vextir_overdtryggt,
    ),
    columns=["Mánuður", "Afborgun", "Vextir", "Heildargreiðsla", "Höfuðstóll"],
).set_index("Mánuður")

plt.plot(jafnar_afborganir_overdtryggt["Afborgun"], label="Afborgun")
plt.plot(jafnar_afborganir_overdtryggt["Vextir"], label="Vextir")
plt.plot(jafnar_afborganir_overdtryggt["Heildargreiðsla"], label="Heildargreiðsla")

plt.xlabel("Mánuður")
plt.ylabel("Upphæð (kr.)")
plt.title("Greiðslubyrði láns með jöfnum afborgunum")
plt.legend()

jafnar_greidslur_overdtryggt = pd.DataFrame(
    jafnar_greidslur(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
        vextir=vextir_overdtryggt,
    ),
    columns=["Mánuður", "Afborgun", "Vextir", "Heildargreiðsla", "Höfuðstóll"],
).set_index("Mánuður")

plt.figure()
plt.plot(jafnar_greidslur_overdtryggt["Afborgun"], label="Afborgun")
plt.plot(jafnar_greidslur_overdtryggt["Vextir"], label="Vextir")
plt.plot(jafnar_greidslur_overdtryggt["Heildargreiðsla"], label="Heildargreiðsla")

plt.xlabel("Mánuður")
plt.ylabel("Upphæð (kr.)")
plt.title("Greiðslubyrði láns með jöfnum greiðslum")
plt.legend()

# Útreikningur

jafnar_afborganir_verdtryggt = pd.DataFrame(
    jafnar_afborganir_vertryggt(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_verdtryggt,
        vextir=vextir_verdtryggt,
        verdbolga=verdbolga,
    ),
    columns=[
        "Mánuður",
        "Afborgun",
        "Vextir",
        "Heildargreiðsla",
        "Verðbætur",
        "Höfuðstóll",
    ],
).set_index("Mánuður")

plt.plot(jafnar_afborganir_verdtryggt["Afborgun"], label="Afborgun")
plt.plot(jafnar_afborganir_verdtryggt["Vextir"], label="Vextir")
plt.plot(jafnar_afborganir_verdtryggt["Heildargreiðsla"], label="Heildargreiðsla")

plt.xlabel("Mánuður")
plt.ylabel("Upphæð (kr.)")
plt.title("Greiðslubyrði verðtryggðs láns með jöfnum afborgunum")
plt.legend()

# jafnar_greidslur_overdtryggt = pd.DataFrame(
#     jafnar_greidslur(
#         lansupphaed=lansupphaed,
#         fjoldi_afborgana=fjoldi_afborgana,
#         vextir=vextir,
#     ),
#     columns=['Mánuður', 'Afborgun', 'Vextir', 'Heildargreiðsla', 'Höfuðstóll'],
# ).set_index('Mánuður')

# plt.figure()
# plt.plot(jafnar_greidslur_overdtryggt['Afborgun'], label='Afborgun')
# plt.plot(jafnar_greidslur_overdtryggt['Vextir'], label='Vextir')
# plt.plot(jafnar_greidslur_overdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

# plt.xlabel('Mánuður')
# plt.ylabel('Upphæð (kr.)')
# plt.title('Greiðslubyrði láns með jöfnum greiðslum')
# plt.legend()